
# Implementing 3 layered radial fiber using Bragg exact

The two layer case works, so the transfer matrix maybe is having the problems

In [ ]:
import numpy as np

from fiberamp.fiber.microstruct.bragg import BraggExact
from step_exact import plotlogf
from ngsolve.webgui import Draw
from ngsolve import CF


In [ ]:
n_air = 1.00027717
n_glass = 1.4388164768221814

In [ ]:
A = BraggExact(ts=[15e-6,.5*15e-6, 15e-6], mats=['air', 'glass', 'air'], 
               ns=[lambda x:n_air, lambda x:n_glass, lambda x:n_air],
              maxhs=[.2,.03, .1], wl=3.6e-6)


In [ ]:
k_low = A.k0 * A.ns[0] * A.scale
k_low

In [ ]:
outer = 'h2'
nu = 1
beta = 26.06899568832816-0.0029172641159476297j

# Do fields in 2 regions via transfer matrix

This is to see if the transfer matrix is correct.  Define v = [1,0] coefficients on region 1, then use transfer matrix to get next regions coefficients.  Plot these fields to check for continuity.

In [ ]:
T0 = A.transfer_matrix(beta, nu, A.rhos[0], A.ns[0], A.ns[1])
T1 = A.transfer_matrix(beta, nu, A.rhos[1], A.ns[1], A.ns[2])

In [ ]:
v = np.array([1, .1, 1, .1], dtype=complex)

In [ ]:
w = T0 @ v
x = T1 @ w

In [ ]:
F0 = A.regional_fields(beta, v, 0, nu=nu)
F1 = A.regional_fields(beta, w, 1, nu=nu)
F2 = A.regional_fields(beta, x, 2, nu=nu)


In [ ]:
Ez = CF([F0['Ez'], F1['Ez'], F2['Ez']])
Ephi = CF([F0['Ephi'], F1['Ephi'], F2['Ephi']])
Er = CF([F0['Er'], F1['Er'], F2['Er']])

Hz = CF([F0['Hz'], F1['Hz'], F2['Hz']])
Hphi = CF([F0['Hphi'], F1['Hphi'], F2['Hphi']])

# Bragg Exact Fields

In [ ]:
Draw(1e1*Ez, A.mesh)

In [ ]:
Draw(Ephi, A.mesh)

# May have issue with Transfer matrix

The z-components are continuous, but not the phi components.  Guess it's time to check the transfer matrix they have in Yeh.  I already checked that I'm calculating the fields correctly from the coefficients, so this is basically all that is left.

Did seemingly find an issue, but it didn't help.

# Try by inverse

In [ ]:
M0L = A.state_matrix(beta, nu, A.rhos[0], A.ns[0])
M0R = A.state_matrix(beta, nu, A.rhos[0], A.ns[1])

T0_inv = np.linalg.inv(M0R) @ M0L

In [ ]:
w = T0_inv @ v

In [ ]:
M1L = A.state_matrix(beta, nu, A.rhos[1], A.ns[1])
M1R = A.state_matrix(beta, nu, A.rhos[1], A.ns[2])

T1_inv = np.linalg.inv(M1R) @ M1L

In [ ]:
x = T1_inv @ w

In [ ]:
F0 = A.regional_fields(beta, v, 0, nu=nu)
F1 = A.regional_fields(beta, w, 1, nu=nu)
F2 = A.regional_fields(beta, x, 2, nu=nu)


In [ ]:
Ez = CF([F0['Ez'], F1['Ez'], F2['Ez']])
Ephi = CF([F0['Ephi'], F1['Ephi'], F2['Ephi']])
Er = CF([F0['Er'], F1['Er'], F2['Er']])

Hz = CF([F0['Hz'], F1['Hz'], F2['Hz']])
Hphi = CF([F0['Hphi'], F1['Hphi'], F2['Hphi']])

# Step Exact Fields

In [ ]:
Draw(Ez, A.mesh)

In [ ]:
Draw(Ephi, A.mesh)

In [ ]:
Draw(Hz, A.mesh)

In [ ]:
Draw(Hphi, A.mesh)

# That works

So that settles it: something is off in the transfer matrix.  The state matrices are right, so ... not sure where the issue is.  Thought I totally dissected it with my sympy exploration.

In [ ]:
np.linalg.norm(T0-T0_inv)


In [ ]:
np.linalg.norm(T1-T1_inv)


# Stubborn!!

I hard coded the state matrix inverse, let's see if it works

In [ ]:
M = A.state_matrix(beta, nu, A.rhos[0], A.ns[0])
Minv = A.state_matrix_inverse(beta, nu, A.rhos[0], A.ns[0])

In [ ]:
M, Minv

In [ ]:
m = Minv @ M


In [ ]:
np.round(m.imag, 15)

In [ ]:
np.round(m.real, 15)

# Okay, hard coded inverse works

Found a few sign errors.  Maybe that's what's happening in the transfer matrix too.

In [ ]:
h = A.state_matrix_inverse(beta, nu, A.rhos[0], A.ns[1]) @ A.state_matrix(beta, nu, A.rhos[0], A.ns[0])

In [ ]:
np.round(h-T0,2 )



# So transfer matrix is correct on block diagonal (2x2)

But off diagonals are wrong.  That at least narrows it.

# Finally got it

It does seem there was an issue in Yeh's off diagonal: we should replace 1/x - 1/y with y/x^2 - 1/y.  I thought I did this before, but maybe I did and there was some other error.  Worth double checking again if we decide to report this that if I sub back to Yeh's in code I get a problem.  Not sure that will happen, so need to check.  When I most recently made a sub I used sympy and substituted (x^2-y^2)/y^3, which is not the same.  Made either a sub error in sympy or sympy did something weird in simplifying?  Likely user error though.